In [ ]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from PIL import Image, UnidentifiedImageError
import os
from torch.utils.data import Dataset, DataLoader
import requests
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision import transforms
from tqdm.notebook import tqdm_notebook as tqdm
from io import BytesIO
import re

In [ ]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
# import requests

# url = 'https://huggingface.co/qantev/trocr-large-spanish/resolve/main/example_1.png'
# image = Image.open(requests.get(url, stream=True).raw).convert("RGB")

processor = TrOCRProcessor.from_pretrained('qantev/trocr-large-spanish')
model = VisionEncoderDecoderModel.from_pretrained('qantev/trocr-large-spanish')
# pixel_values = processor(images=image, return_tensors="pt").pixel_values

# generated_ids = model.generate(pixel_values)
# generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [ ]:
# from transformers import MT5ForConditionalGeneration, T5Tokenizer

# # Load model and tokenizer
# model_name = "google/mt5-large"  # You can also use 'mt5-large'
# tokenizer = T5Tokenizer.from_pretrained(model_name)
# model2 = MT5ForConditionalGeneration.from_pretrained(model_name)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
model.to(device)
# model2.to(device)

In [ ]:
image = Image.open('/kaggle/input/ai-of-god-3/Public_data/test_images/Page_1/L_13.png')
image = image.convert('RGB')

pixel_values = processor(images=image, return_tensors="pt").pixel_values
pixel_values = pixel_values.to(device)

generated_ids = model.generate(pixel_values)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]


print(generated_text)
# print(type(generated_text))

In [ ]:
# # Example of generating corrected text
# input_text = generated_text

# # Tokenize the input text
# input_ids = tokenizer(input_text, return_tensors="pt").input_ids

# # input_ids = input_ids.to(device)

# # Generate corrected text
# outputs = model2.generate(input_ids)
# corrected_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# print(f"Corrected Text: {corrected_text}")

In [ ]:
# Open the grayscale image
img = Image.open('/kaggle/input/ai-of-god-3/Public_data/test_images/Page_1/L_11.png').convert('L')

# Define the threshold value
threshold_value = 160  # Example threshold, you can adjust this

# Apply thresholding
binary_img = img.point(lambda p: 255 if p > threshold_value else 0)

# Convert binary image to RGB (optional)
rgb_img = binary_img.convert('RGB')

# Save or pass this image to your model
rgb_img.save('binary_rgb_image.jpg')


In [ ]:
Image.open('/kaggle/working/binary_rgb_image.jpg')

In [ ]:
image = Image.open('/kaggle/working/binary_rgb_image.jpg')
# image = image.convert('RGB')

pixel_values = processor(images=image, return_tensors="pt").pixel_values
pixel_values = pixel_values.to(device)

generated_ids = model.generate(pixel_values)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]


print(generated_text)
# print(type(generated_text))

In [ ]:
def con(img, threshold_value = 160):
    binary_img = img.point(lambda p: 255 if p > threshold_value else 0)

    # Convert binary image to RGB (optional)
    rgb_img = binary_img.convert('RGB')
    
    return rgb_img

In [ ]:
submission_df = pd.read_excel("/kaggle/input/ai-of-god-3/Public_data/submission.csv.xlsx")
base_dir = '/kaggle/input/ai-of-god-3/Public_data/test_images'
submission_data = []

In [ ]:
def natural_sort_key(s):
    return [int(text) if text.isdigit() else text for text in re.split(r'(\d+)', s)]

In [ ]:
# !pip install language-tool-python

# import language_tool_python

# # Initialize the tool for Spanish
# tool = language_tool_python.LanguageTool('es')

In [ ]:
for folder in sorted(os.listdir(base_dir)):
    folder_path = os.path.join(base_dir, folder)
    page_number = folder.split('_')[-1]
    
    if os.path.isdir(folder_path):
        for image_file in sorted(os.listdir(folder_path), key=natural_sort_key):
            if image_file.endswith('.png'):  
                image_path = os.path.join(folder_path, image_file)
                line_number = image_file.split('_')[-1].split('.')[0]
                formatted_image_id = f'P_{page_number}_L_{line_number}'
                # predicted_text = apply_ocr(image_path)

                # # Incorrect Spanish sentence
                # incorrect_text = predicted_text
                
                # # Check and correct the sentence
                # matches = tool.check(incorrect_text)
                # predicted_text = language_tool_python.utils.correct(incorrect_text, matches)

                image = Image.open(image_path).convert('L')
                
                image = con(image)

                pixel_values = processor(images=image, return_tensors="pt").pixel_values
                
                pixel_values = pixel_values.to(device)

                generated_ids = model.generate(pixel_values)
                generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

                predicted_text = generated_text

#                 incorrect_text = predicted_text
                
#                 # Check and correct the sentence
#                 matches = tool.check(incorrect_text)
#                 predicted_text = language_tool_python.utils.correct(incorrect_text, matches)
                                
                submission_data.append({'unique id': formatted_image_id, 'prediction': predicted_text})
                

In [ ]:
for index, row in submission_df.iterrows():
    matching_prediction = next((pred for pred in submission_data if pred['unique id'] == row['unique id']), None)
    if matching_prediction:
        submission_df.at[index, 'prediction'] = matching_prediction['prediction']  


In [ ]:
submission_df.rename(columns={'unique id': 'unique Id'}, inplace=True)

In [ ]:
submission_df.to_csv('submission.csv', index=False)

print("submission file created successfully!")

In [ ]:
submission_df